In [ ]:
#this code is simulated to run every 10 sec instead of 3600
# values are hard coded just to give an example

import time
from datetime import datetime, timedelta

class Campaign:
    def __init__(self, id, brand_id, dayparting_hours):
        self.id = id
        self.brand_id = brand_id
        self.status = "ACTIVE"  # ACTIVE, PAUSED, OFF
        self.spend_today = 0
        self.spend_this_month = 0
        self.dayparting_hours = dayparting_hours  # List of allowed hours

    def update_status(self, current_hour):
        if self.status == "OFF":
            return  # Already turned off due to budget limits
        if current_hour in self.dayparting_hours:
            self.status = "ACTIVE"
        else:
            self.status = "PAUSED"

class Brand:
    def __init__(self, id, name, daily_budget, monthly_budget):
        self.id = id
        self.name = name
        self.daily_budget = daily_budget
        self.monthly_budget = monthly_budget
        self.current_daily_spend = 0
        self.current_monthly_spend = 0
        self.campaigns = []
        self.last_reset_day = datetime.now().day
        self.last_reset_month = datetime.now().month

    def add_campaign(self, campaign):
        self.campaigns.append(campaign)

    def check_and_update_campaigns(self):
        """Check budget limits and update campaign statuses."""
        current_hour = datetime.now().hour

        if self.current_monthly_spend >= self.monthly_budget:
            self.turn_off_all_campaigns()
            return  # No need to check further if the monthly budget is exhausted

        if self.current_daily_spend >= self.daily_budget:
            self.turn_off_all_campaigns()
        else:
            for campaign in self.campaigns:
                campaign.update_status(current_hour)

    def turn_off_all_campaigns(self):
        for campaign in self.campaigns:
            campaign.status = "OFF"

    def reset_daily_budget(self):
        """Reset daily budget and reactivate campaigns."""
        self.current_daily_spend = 0
        for campaign in self.campaigns:
            campaign.spend_today = 0
        self.restore_campaigns()

    def reset_monthly_budget(self):
        """Reset monthly budget and reactivate campaigns."""
        self.current_monthly_spend = 0
        for campaign in self.campaigns:
            campaign.spend_this_month = 0
        self.restore_campaigns()

    def restore_campaigns(self):
        """Reactivate campaigns based on dayparting hours."""
        current_hour = datetime.now().hour
        for campaign in self.campaigns:
            campaign.update_status(current_hour)

    def spend(self, amount):
        """Distribute spend across active campaigns and stop when budget is exhausted."""
        if self.current_daily_spend >= self.daily_budget or self.current_monthly_spend >= self.monthly_budget:
            return  # Stop spending if budget is already reached

        active_campaigns = [c for c in self.campaigns if c.status == "ACTIVE"]
        if not active_campaigns:
            return  # No active campaigns to allocate spend

        spend_per_campaign = min(amount / len(active_campaigns), self.daily_budget - self.current_daily_spend)

        for campaign in active_campaigns:
            if self.current_daily_spend + spend_per_campaign > self.daily_budget:
                campaign.status = "OFF"  # Turn off if we exceed budget
                continue

            campaign.spend_today += spend_per_campaign
            campaign.spend_this_month += spend_per_campaign
            self.current_daily_spend += spend_per_campaign
            self.current_monthly_spend += spend_per_campaign

    def check_budget_reset(self):
        """Reset daily/monthly budgets at midnight or month start."""
        now = datetime.now()
        if now.day != self.last_reset_day:
            self.reset_daily_budget()
            self.last_reset_day = now.day
            print(f"✅ Reset daily budget for {self.name}")

        if now.month != self.last_reset_month:
            self.reset_monthly_budget()
            self.last_reset_month = now.month
            print(f"✅ Reset monthly budget for {self.name}")

# Example Brands and Campaigns
brands = [
    Brand("1", "Brand A", daily_budget=500, monthly_budget=10000),
    Brand("2", "Brand B", daily_budget=300, monthly_budget=5000)
]

# Example campaigns
brands[0].add_campaign(Campaign("101", "1", dayparting_hours=[9,10,11,12,13,14,15,16,17,18]))
brands[1].add_campaign(Campaign("102", "2", dayparting_hours=[12,13,14,15,16,17,18,19]))

def simulate_ad_spend():
    """Simulate ad spend and budget management."""
    start_time = datetime.now()
    iteration = 0  # Stop after 50 iterations for testing

    while iteration < 50:
        print("\n--- Running Budget Check ---")
        for brand in brands:
            brand.spend(amount=50)  # Simulate ad spend
            brand.check_and_update_campaigns()
            brand.check_budget_reset()  # Ensure daily/monthly resets

            print(f"Brand {brand.name}: Daily Spend {brand.current_daily_spend}/{brand.daily_budget}, Monthly Spend {brand.current_monthly_spend}/{brand.monthly_budget}")
            for campaign in brand.campaigns:
                print(f"  Campaign {campaign.id} Status: {campaign.status}, Spend Today: {campaign.spend_today}")

        time.sleep(10)  # Simulate updates every 10 seconds
        iteration += 1

simulate_ad_spend()
